In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.attraction_constants import *

from packages.attraction.Attraction import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver import ActionChains
# from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.remote.webelement import WebElement

from dotenv import load_dotenv, dotenv_values 
import json
import requests
import google.generativeai as genai
from google.generativeai.types import ContentType
from PIL import Image
from IPython.display import Markdown



c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
"""
Formats a search parameter string for Google Maps by replacing spaces with '+'.

Args:
    searchParam: The search parameter string to format.

Returns:
    The formatted search parameter string with spaces replaced by '+'.
"""
def format_searchParam(searchParam) -> str:
    return searchParam.replace(" ", "+")


In [3]:
'''
Simulates infinite scrolling on a Google Maps search page to retrieve all card elements (appeared in DOM)

Args:
    driver: Selenium WebDriver instance representing the open Google Chrome browser window

Returns:
    List of card element
'''
def GoogleMapsInfiniteScroller(driver:WebDriver) -> list[WebDriver]:
    action = ActionChains(driver)
    isFetchAllCard = False
    wrapper_div = driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]')
    temp_list_card = []
    while not isFetchAllCard:
        scroll_origin = ScrollOrigin.from_element(wrapper_div[0])
        action.scroll_from_origin(scroll_origin, 0, 5000).perform()        
        time.sleep(const.SCROLL_PAUSE_TIME)
        # temp_list_card = driver.find_elements(By.CLASS_NAME, "hfpxzc")
        # ensure that there is no others card element(fetched from google every time scrolling) --> break the loop
        # check whether scorlling untill found section 'คุณมาถึงส่วนท้ายของรายการแล้ว' or 'You've reached the end of the list.'
        reach_end_section_span = driver.find_elements(By.CLASS_NAME, "HlvSq")
        if(len(reach_end_section_span)):
            for cur_span in reach_end_section_span:
                if(cur_span.text == "คุณมาถึงส่วนท้ายของรายการแล้ว" or cur_span.text == "You've reached the end of the list."):
                    isFetchAllCard = True
                    temp_list_card = driver.find_elements(By.CLASS_NAME, "hfpxzc")
                    break
    
    return temp_list_card.copy()

In [4]:
'''
Simulates infinite scrolling on a Google Maps Images page to retrieve all card elements (appeared in DOM)

Args:
    driver: Selenium WebDriver instance representing the open Google Chrome browser window.
    max_len_img: Maximum number of images to collect.

Returns:
    List of image URLs.
'''
def findImageInfiniteScroller(driver:WebDriver, max_len_img:int) -> list[str]:
    res_imgPath = []
    cnt_card = 0

    while(True):
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="play"]/div/button[2]')))
        cur_right_arrow = driver.find_elements(By.XPATH, '//*[@id="play"]/div/button[2]')[0]
        cur_disabled_status = cur_right_arrow.get_attribute('disabled')
        cnt_card += 1
        if(cnt_card >= max_len_img or cur_disabled_status == 'true'):
            break
        cur_right_arrow.click()
    
    time.sleep(3)
    
    # extract image from css property of all element in 'all_list_card'
    all_list_card = driver.find_elements(By.CLASS_NAME, 'Uf0tqf')[:max_len_img+1]
    for cur_card in all_list_card:
        cur_bgImg_val = cur_card.value_of_css_property('background-image')
        match = re.search(r'url\("(.*?)"\)', cur_bgImg_val)
        if match:
            res_imgPath.append(match.group(1))
            
    return res_imgPath.copy() if len(res_imgPath) > 0 else ['']

In [5]:
'''
Gets all images(url) of a given attraction

Args:
    driver: Selenium WebDriver instance representing the open Google Chrome browser window.

Returns:
    List of image URLs.
'''
def findImgPathCurCard(driver:webdriver) -> list[str]:
    options = webdriver.ChromeOptions()
    options.add_argument("start-maximized")

    # get the new URL
    new_cur_card_url = driver.current_url
    # print('new_cur_card_url -> ',new_cur_card_url)
    # create a new WebDriver instance
    new_cur_card_driver = webdriver.Chrome(options=options)
    new_cur_card_driver.get(new_cur_card_url)
    WebDriverWait(new_cur_card_driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'ZKCDEc')))
    # WebDriverWait(new_cur_card_driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'aoRNLd kn2E5e NMjTrf lvtCsd')))
    soup = BeautifulSoup(new_cur_card_driver.page_source, 'html.parser')

    # find length of all reviewed images for current attraction(rendered at hovered thumbnail as '26 รูป', '32,667 รูป')
    len_all_img = 0
    find_img_len = soup.find_all('div', class_='YkuOqf')
    if(len(find_img_len)):
        len_all_img = int(find_img_len[0].text.replace(' รูป','').replace(',',''))
    # in some case there is no reviewed images --> exit process(can not continue to read images)
    else:
        return ['']
    # print('check len_all_img -> ', len_all_img)

    # click at thumnail image button if there's some image of current attraction
    btn_click_thumbnail_temp_1 = new_cur_card_driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[1]/div[1]/button')
    btn_click_thumbnail_temp_2 = new_cur_card_driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[1]/div[1]/button')
    btn_click_thumbnail = btn_click_thumbnail_temp_1.copy() if len(btn_click_thumbnail_temp_1) > 0 else btn_click_thumbnail_temp_2.copy()
    btn_click_thumbnail[0].click()
    
    # ----------------------------------------------------------------------------------
    
    # process of reading at most 20 image
    # wait for page to load (change url from 'current card url' to 'website with all reviewed images')
    WebDriverWait(new_cur_card_driver, 10).until(EC.url_changes(new_cur_card_driver.current_url))

    # get the new URL
    new_readImg_url = new_cur_card_driver.current_url
    # print('new_readImg_url -> ', new_readImg_url)
    # create a new WebDriver instance
    new_readImg_driver = webdriver.Chrome(options=options)
    new_readImg_driver.get(new_readImg_url)
    try:
        WebDriverWait(new_readImg_driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')))
        WebDriverWait(new_readImg_driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'Uf0tqf')))
        WebDriverWait(new_readImg_driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="play"]/div/button[2]'))) 
        
    except Exception as e:
        return ['']
    
    # read the first fetched image
    # then read at most XX images
    max_len_img = min(20, len_all_img)
    res_imgPath = findImageInfiniteScroller(
        driver = new_readImg_driver,
        max_len_img = max_len_img
    )
   
    new_readImg_driver.close()
    time.sleep(1)
    new_cur_card_driver.close()
    time.sleep(1)
    
    return res_imgPath.copy()

In [6]:
"""
Gets tag scores for a given attraction using the Gemini API.

Args:
    name: Name of the attraction.
    latitude: Latitude of the attraction.
    longitude: Longitude of the attraction.
    all_img_url: List of image URLs of the attraction.

Returns:
    Dictionary representing scores for all tags.
"""
def getScorefromGeminiAPI(name:str, latitude:float, longitude:float, all_img_url:list[str]) -> dict:

    # create directory 'temp' to store temporary downloaded images to be used as a request to gemini api
    createDirectory(fh.STORE_ATTRACTION_SCRAPING, 'temp')

    for Idx, cur_url in enumerate(all_img_url):
        if(cur_url == ''):
            break
        response = requests.get(cur_url)
        if response.status_code == 200:
            filename = 'temp/temp_img_{0}.jpeg'.format(Idx)
            with open(filename, 'wb') as file:
                file.write(response.content)

    ## send api to get score for current attraction (send with query and main image)
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    model = genai.GenerativeModel('gemini-1.5-pro-latest')
    text_prompt = "Provide place name, latitude, and longitude. I will return a JSON string containing scores (0-1) for following attributes(nothing else no other sentences)" + \
    "\nfor example: \'{\"Tourism\":0,\"Adventure\":0,\"Meditation\":0,\"Art\":0,\"Cultural\":0,\"Landscape\":0,\"Nature\":0,\"Historical\":0,\"Cityscape\":0,\"Beach\":0,\"Mountain\":0,\"Architecture\":0,\"Temple\":0,\"WalkingStreet\":0,\"Market\":0,\"Village\":0,\"NationalPark\":0,\"Diving\":0,\"Snuggle\":0,\"Waterfall\":0,\"Island\":0,\"Shopping\":0,\"Camping\":0,\"Fog\":0,\"Cycling\":0,\"Monument\":0,\"Zoo\":0,\"Waterpark\":0,\"Hiking\":0,\"Museum\":0,\"Riverside\":0,\"NightLife\":0,\"Family\":0,\"Kid\":0,\"Landmark\":0,\"Forest\":0}" + \
    "\n{0}, {1}, {2} give me score for this".format(name, latitude, longitude)

    # prompt the model
    prompt = [text_prompt]
    for Idx, cur_path_img in enumerate(glob.glob(os.path.join(fh.STORE_ATTRACTION_SCRAPING, 'temp', '*.jpeg'))):
        # prompt with at most 3 images to reduce token usage
        if(Idx == 3):
            break
        cur_img_prompt = Image.open(cur_path_img)
        prompt.append(cur_img_prompt)
    print("total_tokens: ", model.count_tokens(prompt))
    
    res_score_dict = {}
    
    try:
        response = model.generate_content(prompt)
        # remove directory 'temp'
        removeNoneEmptyDir(os.path.join(fh.STORE_ATTRACTION_SCRAPING, 'temp'))
        res_start_Idx = response.text.find('{')
        res_end_Idx = response.text.find('}')
        res_score_dict =  json.loads(response.text[res_start_Idx:res_end_Idx+1])

    except Exception as e:
        # remove directory 'temp'
        removeNoneEmptyDir(os.path.join(fh.STORE_ATTRACTION_SCRAPING, 'temp'))
        print("failed to use gemini api")
    
    return res_score_dict

In [7]:
"""
Extracts data from a given attraction card on Google Maps.

Args:
    driver: Selenium WebDriver instance representing the open Google Chrome browser window.
    cur_card: The current card element.
    cur_attraction: The current attraction object.
    cur_province_th: The current province name in Thai language.
    cur_geo_code_by_province_df: Dataframe containing geocodes for the given province.

Returns:
    Dataframe containing the scraped data from the given card.
"""
def getDatafromGoogleMapCard(driver:webdriver, cur_card:WebElement, cur_attraction:Attraction, cur_province_th:str, cur_district_th:str, cur_geo_code_by_province_df:pd.DataFrame) -> pd.DataFrame:
    wait = WebDriverWait(driver, 10)
    action = ActionChains(driver)

    patternPhone = re.compile(r'\d{3} \d{3} \d{4}')
    patternPhoneService = re.compile(r'\d{3} \d{3} \d{3}')

    subStrDistrict = "อำเภอ"
    subStrSubDistrict = "ตำบล"

    if cur_province_th == "กรุงเทพมหานคร":
        subStrDistrict = "เขต"
        subStrSubDistrict = "แขวง"

    try:
        action.move_to_element(cur_card).perform()
        cur_card.click()
        
    except Exception as e:
        # print('can not click card element')
        # print('when scrape for province -> {0}'.format(cur_province_th))
        return

    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    name = soup.find_all('h1', class_='DUwDvf lfPIob')
    description = soup.find_all('div', class_='PYvSYb')
    address = soup.find_all('div', class_='Io6YTe')
    # loc = soup.find_all('div', class_='rogA2c')

    time.sleep(5)
    start_index_lat = driver.current_url.find("!3d") + 3
    end_index_lat = driver.current_url.find("!4d")
    lat = driver.current_url[start_index_lat:end_index_lat]
    start_index_long = driver.current_url.find("!4d") + 3
    end_index_long = driver.current_url.find("!", driver.current_url.find("!4d") + 1)
    long = driver.current_url[start_index_long:end_index_long]
    
    # found some wiered place that doesn't even have its address
    # skip this case for now...
    if(not len(address)):
        return pd.DataFrame()

    district = 0
    subDirstrict = 0

    # find attraction name
    print('name: ',name[0].text)
    cur_attraction.set_name(name[0].text)

    # find lat, long
    print("lat, long: " + lat + " " + long)
    cur_attraction.set_latitude(float(lat))
    cur_attraction.set_longitude(float(long))


    # find description
    if(len(description)):
        # print("Description :",description[0].text)
        cur_attraction.set_description(description[0].text)
    

    # find location
    useData = None
    for div in address:
        if cur_province_th in div.text and div.text.find(subStrDistrict) != -1:
            useData = div.text.replace(",","").replace("เเ","แ")
    
    if(useData != None):
        # print("Full Address :",useData)
        # another brute force way in case of province 'กรุงเทพหมานคร' not have word 'แขวง' in address
        if(cur_province_th == 'กรุงเทพมหานคร' and useData.find(subStrSubDistrict) == -1):
            subAddress_split = useData.split(' ')
            cur_province_Idx = subAddress_split.index(cur_province_th)
            district = subAddress_split[cur_province_Idx - 1].replace("เขต","")
            subDistrict = subAddress_split[cur_province_Idx - 2].replace("แขวง","")

        else:
            start_address_index = useData.find(subStrSubDistrict)
            subAddress = useData[start_address_index:]
            district = subAddress[subAddress.find(subStrDistrict)+len(subStrDistrict):subAddress.find(cur_province_th)].replace(" ","")               
            subDistrict = subAddress[subAddress.find(subStrSubDistrict)+len(subStrSubDistrict):subAddress.find(subStrDistrict)].replace(" ","")

        if district == "เมือง":
            district = district+cur_province_th

        # skip if 'district' not matched with 'cur_district_th'
        if(district != cur_district_th):
            return

        # filter row to find 'ISO_3166_code', 'zip_code', 'geo_code'
        filtered_rows = cur_geo_code_by_province_df[
            (cur_geo_code_by_province_df['district_th'] == district) & (cur_geo_code_by_province_df['subDistrict_th'] == subDistrict)
        ]
        filtered_rows.reset_index(inplace=True, drop=True)
        if not filtered_rows.empty:
            # print("province :",filtered_rows.loc[0, 'ISO_3166_code'],cur_province_th)
            # print("District :",filtered_rows.loc[0, 'zip_code'],district)
            # print("SubDistrict :",filtered_rows.loc[0, 'geo_code'],subDistrict)

            cur_attraction.set_location(
                address = useData,
                province = cur_province_th,
                district = district,
                sub_district = subDistrict,
                iso_code = filtered_rows.loc[0, 'ISO_3166_code'],
                zip_code = filtered_rows.loc[0, 'zip_code'],
                geo_code = filtered_rows.loc[0, 'geo_code']
            )
        else:
            # print("province :",cur_province_th)
            # print("District :",district)
            # print("SubDistrict :",subDistrict)

            cur_attraction.set_location(
                address = useData,
                province = cur_province_th,
                district = district,
                sub_district = subDistrict,
                iso_code = 0,
                zip_code = 0,
                geo_code = 0
            )

    # find rating
    score_div = soup.find_all('span', class_='ceNzKf')
    if len(score_div):
        score = score_div[0].get('aria-label').replace(" ","").replace("ดาว","").replace(",", "")
        review_count_div = driver.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div[2]/div/div[1]/div[2]/span[2]/span/span')
        review_count = review_count_div[0].text.replace("(", "").replace(")", "").replace(",", "")
        
        # print("Rating score :",score)
        # print("Rating Count:", review_count)
      
        cur_attraction.set_rating(
            score = float(score),
            rating_count = int(review_count)
        )
        
    
    # find ticket rate
    ticketRating = soup.find_all('div', class_='drwWxc')
    if(len(ticketRating)):
        # print("Ticket Price :",ticketRating[0].text)
        pass


    # find phone number
    divContact = soup.find_all('div', class_='Io6YTe fontBodyMedium kR99db')
    for div in divContact:
        if(re.match(patternPhone,div.text) or re.match(patternPhoneService,div.text)):
            # print("Contact :",div.text)
            cur_attraction.set_phone(div.text)


    # find website
    divWebsite = soup.find_all('a', class_='CsEnBe')
    if(len(divWebsite)):
        for i in range(len(divWebsite)):
            # check if it fit below case
            website_href = divWebsite[i].get('href')
            if(website_href):
                cur_attraction.set_website(website_href)
                break


    # find opening hours
    openingHourCheck = soup.find_all("span", class_="HlvSq")

    if(len(openingHourCheck) and openingHourCheck[0].text == "ดูเวลาทำการเพิ่มเติม"):
        infoOpening = driver.find_elements(By.CLASS_NAME, "HlvSq")
        for element in infoOpening:
            element.click()
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'y0skZc')))
        soup = BeautifulSoup(driver.page_source, 'html.parser')

    openingTime = soup.find_all("tr", class_="y0skZc")

    '''
    update 28/7/2024: found this case in place 'พิพิธภัณฑ์ศิลปะไทยร่วมสมัย', 'วิหารเทพวิทยาคม'
    
    วันอาทิตย์ (วันเฉลิมพระชนมพรรษา พระบาทสมเด็จพระปรเมนทรรามาธิบดีศรีสินทรมหาวชิราลงกรณ พระวชิรเกล้าเจ้าอยู่หัว)                10:00–18:00เวลาทำการในวันหยุด
    วันจันทร์ (วันเฉลิมพระชนมพรรษา พระบาทสมเด็จพระปรเมนทรรามาธิบดีศรีสินทรมหาวชิราลงกรณ พระวชิรเกล้าเจ้าอยู่หัว (วันหยุดชดเชย))    ปิดทำการเวลาทำการในวันหยุด
    วันอังคาร      10:00–18:00
    วันพุธ        10:00–18:00
    วันพฤหัสบดี    10:00–18:00
    วันศุกร์       10:00–18:00
    วันเสาร์         10:00–18:00
    '''

    cur_openingHour = {}
    for data in openingTime:
        dateDiv = data.find("td", class_="ylH6lf")
        timeDiv = data.find("td", class_="mxowUb")
        # print(dateDiv.text,timeDiv.text)
        cur_openingHour[dateDiv.text] = timeDiv.text
        
    cur_attraction.set_openingHour(cur_openingHour)


    # find img path
    cur_card_img_path = findImgPathCurCard(
        driver = driver
    )
    # print("check res imgPath arr -> ",cur_card_img_path)
    cur_attraction.set_imgPath(cur_card_img_path)

    # find tag score using Gemini API
    try:
        all_tag_score = getScorefromGeminiAPI(
            name = cur_attraction.get_name(),
            latitude = cur_attraction.get_latitude(),
            longitude = cur_attraction.get_longitude(),
            all_img_url = cur_attraction.get_imgPath()
        )
        # set tag score to Attraction object
        for cur_tag in all_tag_score.keys():
            cur_attraction.set_tag_score(key=cur_tag, val=all_tag_score[cur_tag])
            
    except Exception as e:
        pass

    # create result data frame (dataframe represent data read from current card element)
    res_dict = {
        'name' : [cur_attraction.get_name()],
        'description' : [cur_attraction.get_description()],
        'latitude' : [cur_attraction.get_latitude()],
        'longitude' : [cur_attraction.get_longitude()],
        'imgPath' : [cur_attraction.get_imgPath()],
        'phone': [cur_attraction.get_phone()],
        'website': [cur_attraction.get_website()],
        'openingHour': [cur_attraction.get_openingHour()],

        # location
        'address' : [cur_attraction.get_location().get_address()],
        'province' : [cur_attraction.get_location().get_province()],
        'district' : [cur_attraction.get_location().get_district()],
        'subDistrict' : [cur_attraction.get_location().get_subDistrict()],
        'ISO_3166_code' : [cur_attraction.get_location().get_ISO_3166_code()],
        'zip_code' : [cur_attraction.get_location().get_zip_code()],
        'geo_code' : [cur_attraction.get_location().get_geo_code()],

        # rating
        'score' : [cur_attraction.get_rating().get_score()],
        'ratingCount' : [cur_attraction.get_rating().get_ratingCount()],
    }

    # append attraction tags score in 'dict'
    for cur_tag in ATTRACTION_TAG_SCORE:
        res_dict[cur_tag] = cur_attraction.get_attractionTag().get_tag_score(cur_tag)

    # create result dataframe
    res_df = pd.DataFrame(res_dict)
    
    return res_df.copy()

In [8]:
# loading variables from .env file
load_dotenv() 

geo_code_df = pd.read_csv(fh.PATH_TO_GEOCODE)

cnt_for_test = 1

# create directory 'res_attraction_scraping'
createDirectory(fh.STORE_ATTRACTION_SCRAPING, 'res_attraction_scraping')

cnt_test_province = 0
for cur_province_th, cur_province_en in zip(ALL_PROVINCE_TH, ALL_PROVINCE_ENG):
    # for testing workflow
    if(cnt_test_province == 2):
        break
    cnt_test_province += 1
    
    cur_province_data_df = pd.DataFrame()

    cur_geo_code_by_province_df = geo_code_df[geo_code_df['province_en'] == cur_province_en]
    cur_geo_code_by_province_df.drop(columns=['province_th', 'province_en'], inplace=True)
    cur_all_searchDistrict_th = np.unique(cur_geo_code_by_province_df.district_th.to_list(), axis=0)
    cur_all_searchDistrict_en = np.unique(cur_geo_code_by_province_df.district_en.to_list(), axis=0)
    
    cnt_for_test_distirct = 0
    for cur_search_district_th, cur_search_district_en in zip(cur_all_searchDistrict_th, cur_all_searchDistrict_en):
        print("cur_province -> {0}, cur_distrinct -> {1}".format(cur_province_en, cur_search_district_en))
        # for testing workflow
        if(cnt_for_test_distirct == 1):
            break
        cnt_for_test_distirct += 1        
        
        # สถาน ที่ ท่องเที่ยว near Mueang Nakhon Ratchasima District, Nakhon Ratchasima
        cur_searchParam = format_searchParam(
            searchParam="สถาน ที่ ท่องเที่ยว near %s District, %s" % (cur_search_district_en, cur_province_en)
        )
        cur_query_url = "https://www.google.com/maps/search/%s" % (cur_searchParam)
        
        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        # create webdriver instance using option to mazimize current window
        driver = webdriver.Chrome(options=options)
        driver.get(cur_query_url)
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'hfpxzc')))
        list_card = driver.find_elements(By.CLASS_NAME, "hfpxzc")
        
        print("check listCard 1")
        print(len(list_card))
        print(list_card)
        print("**************************")

        # Simulates infinite scrolling on a Google Maps search page to retrieve all card elements in the next step
        list_card = GoogleMapsInfiniteScroller(
            driver = driver,
        )

        print("check listCard 2")
        print("nnbm. -> ",driver.current_url)
        print(len(list_card))
        print(list_card)
        print("**************************")

        # after google map have fetched all card elements
        # read data from card elements
        cur_district_df = pd.DataFrame()
        for i in range(len(list_card)):
            # print("cur Idx -> ", i)
            # for debug puepose
            # if(i == 5):
            #     break
            # define attraction object
            cur_attraction = Attraction()
            cur_card_df = getDatafromGoogleMapCard(
                driver = driver,
                cur_card = list_card[i],
                cur_attraction = cur_attraction,
                cur_province_th = cur_province_th,
                cur_district_th=cur_search_district_th,
                cur_geo_code_by_province_df = cur_geo_code_by_province_df.copy() 
            )
            # concat all data frame of current distrcit to 'cur_province_data_df'
            cur_district_df = pd.concat([cur_district_df, cur_card_df])

        # concat all data frame of current province, distrcit to 'cur_province_data_df'
        cur_province_data_df = pd.concat([cur_province_data_df, cur_district_df]) 

        # close current browser tab    
        driver.close()  
    
    # remove duplicate attraction name of current province, distrcit, sub_district
    cur_province_data_df.drop_duplicates(subset=['name', 'ISO_3166_code', 'zip_code', 'geo_code'], inplace=True)

    # set new index
    cur_province_data_df.set_index(['name', 'province', 'district', 'subDistrict'], inplace=True)

    # save data to csv
    # which migth be used for POST API save to database later...
    res_file_name = 'res_attraction_{0}.csv'.format(cur_province_en)
    res_path = os.path.join(fh.STORE_ATTRACTION_SCRAPING, 'res_attraction_scraping', res_file_name) 
    cur_province_data_df.to_csv(res_path)

C:\Users\user\AppData\Local\Temp\ipykernel_8836\4105860966.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_geo_code_by_province_df.drop(columns=['province_th', 'province_en'], inplace=True)


Directory res_attraction_scraping created successfully
cur_province -> Phuket, cur_distrinct -> Kathu
check listCard 1
6
[<selenium.webdriver.remote.webelement.WebElement (session="69e98f2e4fc9bfd688010ead86118cf4", element="f.F97FA3631B1FE658B560AA19896CF69B.d.9E5EF2CE428332D829E42E709A7D5229.e.67")>, <selenium.webdriver.remote.webelement.WebElement (session="69e98f2e4fc9bfd688010ead86118cf4", element="f.F97FA3631B1FE658B560AA19896CF69B.d.9E5EF2CE428332D829E42E709A7D5229.e.68")>, <selenium.webdriver.remote.webelement.WebElement (session="69e98f2e4fc9bfd688010ead86118cf4", element="f.F97FA3631B1FE658B560AA19896CF69B.d.9E5EF2CE428332D829E42E709A7D5229.e.69")>, <selenium.webdriver.remote.webelement.WebElement (session="69e98f2e4fc9bfd688010ead86118cf4", element="f.F97FA3631B1FE658B560AA19896CF69B.d.9E5EF2CE428332D829E42E709A7D5229.e.70")>, <selenium.webdriver.remote.webelement.WebElement (session="69e98f2e4fc9bfd688010ead86118cf4", element="f.F97FA3631B1FE658B560AA19896CF69B.d.9E5EF2CE42

Exception ignored in: <function Service.__del__ at 0x000002861843AD40>
Traceback (most recent call last):
  File "c:\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 131, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 120, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\selenium\webdriver\common\utils.py", line 101, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\socket.py", line 844, i

Directory temp created successfully
total_tokens:  total_tokens: 994



WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=127.0.6533.100)
Stacktrace:
	GetHandleVerifier [0x00007FF600379632+30946]
	(No symbol) [0x00007FF60032E3C9]
	(No symbol) [0x00007FF600226FDA]
	(No symbol) [0x00007FF60020DB35]
	(No symbol) [0x00007FF60020E46F]
	(No symbol) [0x00007FF6002275F3]
	(No symbol) [0x00007FF6001FC67D]
	(No symbol) [0x00007FF6002AA7C1]
	(No symbol) [0x00007FF60029CA30]
	(No symbol) [0x00007FF60026A6E5]
	(No symbol) [0x00007FF60026B021]
	GetHandleVerifier [0x00007FF6004AF83D+1301229]
	GetHandleVerifier [0x00007FF6004BBDB7+1351783]
	GetHandleVerifier [0x00007FF6004B2A03+1313971]
	GetHandleVerifier [0x00007FF6003ADD06+245686]
	(No symbol) [0x00007FF60033758F]
	(No symbol) [0x00007FF600333804]
	(No symbol) [0x00007FF600333992]
	(No symbol) [0x00007FF60032A3EF]
	BaseThreadInitThunk [0x00007FFFA700257D+29]
	RtlUserThreadStart [0x00007FFFA7B0AF28